In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split, WeightedRandomSampler, SubsetRandomSampler
import os
from tqdm import tqdm
from PIL import Image
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class GalaxyDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        
        # Store all labels for sampling weights calculation
        self.labels = []
        for img_name in self.image_files:
            label_str = img_name.split('_')[-1].split('.')[0]
            self.labels.append(int(label_str))
        
        # Calculate class weights
        label_counter = Counter(self.labels)
        self.class_weights = {cls: 1.0/count for cls, count in label_counter.items()}
        
        # Store sample weights for WeightedRandomSampler
        self.sample_weights = [self.class_weights[label] for label in self.labels]
        self.sample_weights = torch.FloatTensor(self.sample_weights)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.root_dir, img_name)
        
        label = self.labels[idx]
        
        # Load and preprocess image
        image = Image.open(img_path).convert("RGB")
        image = image.resize((128, 128))
        
        # Data augmentation for training
        if torch.rand(1) > 0.5:  # Random horizontal flip
            image = image.transpose(Image.FLIP_LEFT_RIGHT)
        
        # Convert to numpy array and normalize
        image_array = np.array(image) / 255.0
        
        # Convert to tensor
        image_tensor = torch.FloatTensor(image_array).permute(2, 0, 1)
        
        return image_tensor, label

class GalaxyCNN(nn.Module):
    def __init__(self, num_classes):
        super(GalaxyCNN, self).__init__()
        
        self.features = nn.Sequential(
            # First block
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Second block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Third block
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Fourth block
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),  # Reduced dropout
            nn.Linear(512 * 8 * 8, 2048),  # Smaller first dense layer
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),  # Reduced dropout
            nn.Linear(2048, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)
        )
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0.01):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf

    def __call__(self, val_loss):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score - self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, early_stopping, num_epochs, fold=None):
    model.to(device)
    best_val_acc = 0.0
    train_losses, val_losses = [], []
    fold_str = f" (Fold {fold})" if fold is not None else ""

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}{fold_str}')
        
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            progress_bar.set_postfix({'loss': running_loss / len(train_loader), 'accuracy': 100. * correct / total})
        
        train_losses.append(running_loss / len(train_loader))
        
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = 100. * val_correct / val_total
        val_losses.append(val_loss)
        
        print(f'\nEpoch {epoch+1}/{num_epochs}{fold_str}: Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')
        
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping triggered{fold_str}")
            break
        
        scheduler.step(val_accuracy)
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            if fold is not None:
                torch.save(model.state_dict(), f'best_model_fold{fold}.pth')
            else:
                torch.save(model.state_dict(), 'best_model.pth')

    if fold is not None:
        plot_loss(train_losses, val_losses, fold)
    else:
        plot_loss(train_losses, val_losses)
        
    return best_val_acc

def plot_loss(train_losses, val_losses, fold=None):
    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    title = 'Training & Validation Loss'
    if fold is not None:
        title += f' (Fold {fold})'
    plt.title(title)
    plt.savefig(f'loss_plot{"_fold" + str(fold) if fold is not None else ""}.png')
    plt.close()

def evaluate_model(model, loader, num_classes=10, fold=None):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc=f'Evaluating{"" if fold is None else " (Fold " + str(fold) + ")"}'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100. * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    
    conf_matrix = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=range(num_classes), yticklabels=range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    title = 'Confusion Matrix'
    if fold is not None:
        title += f' (Fold {fold})'
    plt.title(title)
    plt.savefig(f'confusion_matrix{"_fold" + str(fold) if fold is not None else ""}.png')
    plt.close()
    
    print("\nClassification Report:\n", classification_report(all_labels, all_preds, digits=4))

    return accuracy

def k_fold_cross_validation(dataset, num_folds=5, num_classes=10, num_epochs=30):
    # First, check that the dataset is loaded properly
    print(f"Dataset contains {len(dataset)} samples")
    if len(dataset) == 0:
        print("ERROR: Dataset is empty! Check the data directory path.")
        return 0
    
    # Print some sample filenames to verify
    print(f"Sample image filenames: {dataset.image_files[:3]}")
    
    # Initialize k-fold cross validation
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    
    # Store accuracy for each fold
    fold_accuracies = []
    
    # Create indices for KFold to use
    indices = list(range(len(dataset)))
    
    # Loop through each fold
    for fold, (train_ids, val_ids) in enumerate(kfold.split(indices)):
        print(f"\n{'='*20} FOLD {fold+1}/{num_folds} {'='*20}")
        
        # Create data samplers for train and validation sets
        train_sampler = SubsetRandomSampler(train_ids)
        val_sampler = SubsetRandomSampler(val_ids)
        
        # Create data loaders for current fold
        train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
        val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)
        
        # Initialize model, criterion, optimizer, and early stopping for current fold
        model = GalaxyCNN(num_classes=num_classes).to(device)
        
        # Get class weights for criterion from the dataset
        class_weights = torch.FloatTensor([1.0/1081, 1.0/1853, 1.0/2645, 1.0/2027, 1.0/334, 
                                          1.0/2043, 1.0/1829, 1.0/2628, 1.0/1423, 1.0/1873]).to(device)
        criterion = nn.CrossEntropyLoss(weight=class_weights)
        optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
        early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.01)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True, min_lr=1e-6)
        
        # Train and evaluate model for current fold
        train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, early_stopping, num_epochs, fold+1)
        accuracy = evaluate_model(model, val_loader, num_classes, fold+1)
        fold_accuracies.append(accuracy)
        
        # Save model for current fold
        torch.save(model.state_dict(), f"galaxy_cnn_fold{fold+1}.pth")
        
    # Print average and per-fold accuracies
    print("\n" + "="*50)
    print(f"K-Fold Cross-Validation Results for {num_folds} Folds")
    print("="*50)
    print(f"Average Accuracy: {np.mean(fold_accuracies):.2f}% (±{np.std(fold_accuracies):.2f}%)")
    print("\nPer-fold Accuracy:")
    for fold, accuracy in enumerate(fold_accuracies):
        print(f"Fold {fold+1}: {accuracy:.2f}%")
    print("="*50)
    
    # Return average accuracy
    return np.mean(fold_accuracies)

# Dataset path
data_dir = os.path.expanduser("Decals_data/Decals_data_images")

# Check if data directory exists
if not os.path.exists(data_dir):
    print(f"ERROR: Data directory '{data_dir}' does not exist!")
    print("Please check the path and try again.")
    # You could add code to exit here if needed
else:
    print(f"Data directory found: {data_dir}")
    # List first few items to verify
    files = os.listdir(data_dir)[:5]
    print(f"First few files in directory: {files}")

# Create dataset
dataset = GalaxyDataset(root_dir=data_dir)

# Print dataset information
print(f"Dataset loaded with {len(dataset)} images")
print(f"Class distribution: {Counter(dataset.labels)}")

# Perform 5-fold cross-validation
num_classes = 10  # Based on your class weights, seems like 10 classes
num_epochs = 30    # Same as original
k_fold_cross_validation(dataset, num_folds=5, num_classes=num_classes, num_epochs=num_epochs)

# After finding the best fold, you might want to train on the entire dataset
print("\nTraining final model on the entire dataset...")
# Create data loader for full dataset
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize model, criterion, optimizer, and early stopping
model = GalaxyCNN(num_classes=num_classes).to(device)
class_weights = torch.FloatTensor([1.0/1081, 1.0/1853, 1.0/2645, 1.0/2027, 1.0/334, 
                                  1.0/2043, 1.0/1829, 1.0/2628, 1.0/1423, 1.0/1873]).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True, min_lr=1e-6)

# We need a small validation set for the scheduler and early stopping
val_size = int(0.1 * len(dataset))
train_size = len(dataset) - val_size
temp_train_dataset, temp_val_dataset = random_split(dataset, [train_size, val_size])
val_loader = DataLoader(temp_val_dataset, batch_size=32, shuffle=False)

# Train final model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, early_stopping, num_epochs)

# Save final model
torch.save(model.state_dict(), "galaxy_cnn_final.pth")
print("Final model saved successfully!")

Using device: cuda
Data directory found: Decals_data/Decals_data_images
First few files in directory: ['image_9483_5.png', 'image_15099_8.png', 'image_9226_5.png', 'image_16911_9.png', 'image_3980_2.png']
Dataset loaded with 17736 images
Class distribution: Counter({2: 2645, 7: 2628, 5: 2043, 3: 2027, 9: 1873, 1: 1853, 6: 1829, 8: 1423, 0: 1081, 4: 334})
Dataset contains 17736 samples
Sample image filenames: ['image_9483_5.png', 'image_15099_8.png', 'image_9226_5.png']

==================== FOLD 1/5 ====================


/root/miniconda3/envs/myenv/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/30 (Fold 1): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 444/444 [01:06<00:00,  6.67it/s, loss=2.04, accuracy=24.6]



Epoch 1/30 (Fold 1): Validation Loss: 1.8693, Accuracy: 27.79%


Epoch 2/30 (Fold 1): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 444/444 [01:06<00:00,  6.71it/s, loss=1.73, accuracy=33.6]



Epoch 2/30 (Fold 1): Validation Loss: 1.5027, Accuracy: 42.02%


...

Epoch 23/30 (Fold 3): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 444/444 [01:06<00:00,  6.73it/s, loss=0.397, accuracy=85.4]



Epoch 23/30 (Fold 3): Validation Loss: 0.7097, Accuracy: 79.19%


Epoch 24/30 (Fold 3): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 444/444 [01:04<00:00,  6.87it/s, loss=0.381, accuracy=86.3]



Epoch 24/30 (Fold 3): Validation Loss: 0.6866, Accuracy: 79.56%


Epoch 25/30 (Fold 3):  25%|█████████████████████████                                                                           | 111/444 [00:16<00:48,  6.92it/s, loss=0.0918, accuracy=86.6]